# Homework I

Diogo Correia (ist199211) & Tomás Esteves (ist199341)

## I. Pen and Paper [12v]

**Given the following decision tree learnt from 20 observation using Shannon entropy, with leaf annotations (`#correct/#total`)**

![Decision Tree](./decision_tree.png)

### 1) [4v] Draw the training confusion matrix

<table>
  <tr>
    <td colspan="2" rowspan="2" style="border-top: none; border-left: none;"></td>
    <th colspan="2">True</th>
    <td rowspan="2" style="border-top: none; border-right: none;"></td>
  </tr>
  <tr>
    <th>Positive</th>
    <th>Negative</th>
  </tr>
  <tr>
    <th rowspan="2">Predicted</th>
    <th>Positive</th>
    <td>8</td>
    <td>4</td>
    <td>12</td>
  </tr>
  <tr>
    <th>Negative</th>
    <td>3</td>
    <td>5</td>
    <td>8</td>
  </tr>
  <tr>
    <th colspan="2" style="border-left: none; border-bottom: none;"></th>
    <td>11</td>
    <td>9</td>
    <td>20</td>
  </tr>
</table>

### 2) [3v] Identify the training F1 after a post-pruning of the given tree under a maximum depth of 1.

<table>
  <tr>
    <td colspan="2" rowspan="2" style="border-top: none; border-left: none;"></td>
    <th colspan="2">True</th>
    <td rowspan="2" style="border-top: none; border-right: none;"></td>
  </tr>
  <tr>
    <th>Positive</th>
    <th>Negative</th>
  </tr>
  <tr>
    <th rowspan="2">Predicted</th>
    <th>Positive</th>
    <td>5</td>
    <td>2</td>
    <td>7</td>
  </tr>
  <tr>
    <th>Negative</th>
    <td>6</td>
    <td>7</td>
    <td>13</td>
  </tr>
  <tr>
    <th colspan="2" style="border-left: none; border-bottom: none;"></th>
    <td>11</td>
    <td>9</td>
    <td>20</td>
  </tr>
</table>

In [ ]:
true_positives = 5
false_positives = 2
false_negatives = 6

precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)

f1_measure = (0.5 * (1 / precision + 1 / recall)) ** (-1)

f1_measure

### 3) [2v] Identify two different reasons as to why the left tree path was not further decomposed.

The left tree path might not have been further decomposed because:

- We did not want to overfit the model, since we have a very small sample size.
  For this reason, if we were to further decompose the left tree path, we might end up with a less accurate
  decision tree, since the 2 negative observations might have been outliers.
- The information gain of this branch, $IG(y_{out} | y_2, y_1 = A)$, might be very small,
  since there are a lot more observations classified as positive than as negative.
  If we were to decompose the left path, there might be no optimal division that would correctly identify all observations.

### 4) [3v] Compute the information gain of variable y1

In [ ]:
from math import log2
import operator as op
from itertools import chain
from functools import reduce

In [ ]:
# INPUT
total_positive_count = 11
total_negative_count = 9

branch_a_positive_count = 5
branch_a_negative_count = 2

branch_b_positive_count = 6
branch_b_negative_count = 7

In [ ]:
# Functions
def entropy_by_count(counts):
    """
    Calculates the information entropy, I(X), of a set, given the count of each class of element
    """
    total = sum(counts)
    return reduce(op.add, map(lambda x: -(x / total) * log2(x / total), counts))


def split_entropy_by_count(branch_counts):
    """
    Calculates the entropy after branching on a variable
    """
    # branch counts is a list of int lists
    total = sum(chain(*branch_counts))
    return reduce(
        op.add, map(lambda x: (sum(x) / total) * entropy_by_count(x), branch_counts)
    )

In [ ]:
entropy_y_out = entropy_by_count([total_positive_count, total_negative_count])
entropy_y_out_y1 = split_entropy_by_count(
    [
        [branch_a_positive_count, branch_a_negative_count],
        [branch_b_positive_count, branch_b_negative_count],
    ]
)

information_gain = entropy_y_out - entropy_y_out_y1

information_gain

## Programming [8v]

**Considering the `pd_speech.arff` dataset available at the homework tab:**

### 1) [6v]

**Using sklearn, apply a stratified 70-30 training-testing split with a fixed seed
(`random_state=1`), and assess in a single plot the training and testing accuracies of a decision tree
with no depth limits (and remaining default behavior) for a varying number of selected features
in `{5,10,40,100,250,700}`. Feature selection should be performed before decision tree learning
considering the discriminative power of the input variables according to mutual information
criterion (`mutual_info_classif`).**

In [ ]:
from operator import itemgetter
import pandas as pd
from scipy.io.arff import loadarff
from sklearn import feature_selection, model_selection, tree, metrics, preprocessing
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Reading the ARFF file
data = loadarff("data/pd_speech.arff")
df = pd.DataFrame(data[0])
df["class"] = df["class"].str.decode("utf-8")

In [ ]:
# Separate features from the outcome (class)
X = df.drop("class", axis=1)
y = df["class"]

In [ ]:
# Split the dataset into a training set (70%) and a testing set (30%)
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, train_size=0.7, stratify=y, random_state=1
)

In [ ]:
NUM_FEATURES = [5, 10, 40, 100, 250, 700]

training_accurancy = []
test_accurancy = []

for num_features in NUM_FEATURES:
    # Select the features with the greatest information gain (by mutual_info_classif)
    kbest = feature_selection.SelectKBest(
        score_func=lambda a, b: feature_selection.mutual_info_classif(
            a, b, random_state=1
        ),
        k=num_features,
    )
    kbest.fit(X_train, y_train)

    # Get the observation sets with only the selected N best features
    X_train_cut = kbest.transform(X_train)
    X_test_cut = kbest.transform(X_test)

    # Fit the decision tree classifier
    predictor = tree.DecisionTreeClassifier(random_state=1)
    predictor.fit(X_train_cut, y_train)

    # Use the decision tree to predict the outcome of the given observations
    y_train_pred = predictor.predict(X_train_cut)
    y_test_pred = predictor.predict(X_test_cut)

    # Get the accuracy of each test
    train_acc = metrics.accuracy_score(y_train, y_train_pred)
    test_acc = metrics.accuracy_score(y_test, y_test_pred)

    training_accurancy.append(train_acc)
    test_accurancy.append(test_acc)

In [ ]:
plt.plot(
    NUM_FEATURES,
    training_accurancy,
    label="Training Accuraccy",
    marker="+",
    color="#4caf50",
)
plt.plot(
    NUM_FEATURES, test_accurancy, label="Test Accuraccy", marker=".", color="#ff5722"
)

plt.xlabel("Number of Selected Features")
plt.ylabel("Accuracy")

plt.legend()
plt.savefig("../../report/assets/hw1-plot.svg")
plt.show()

### 2) [2v]

**Why training accuracy is persistently 1? Critically analyze the gathered results.**

From the obtained results, we noticed that the training accuracy is always 1, regardless of the number of selected features.
This is a result of how decision trees learn.

Since the question prompt tells us the decision tree does not have a depth limit, a decision tree that perfectly fits all the training data (`X_train`) can be created.
Therefore, after the tree is trained, if we give the training set (`X_train`) as the data set to test its accuracy, it'll know the correct path for all of the observations and knows how to classify them.
This results in an accuracy of 1.

However, if we test the model with a data set that it hasn't been trained on (`X_test`), we see its accuracy slightly decreases to around 0.8.
This happens because it has never seen those observations before, so it might have leaves that are not expanded enough to accurately classify them.

Furthermore, we can also notice that the accuracy of the decision tree changes with the number of features.
Using a lot of features to train the model can produce an overfitted tree, reducing its accuracy when predicting the outcome of a new observation,
while using only a few features might not be enough information to train the decision tree.
It's important to find the right features to include in the decision tree, and we can see that both `N = 40` and `N = 250` are good candidates for the number of features to improve, although not by much.